In [35]:

def get_line_number(file_path):
    sys.stderr.write("Counting line number of {}".format(file_path))
    f = gzip.open(file_path)
    lines = f.readlines()
    count =0
    
    for line in lines:
        count += 1
    return count

def parse(filename, total):
    IGNORE_FIELDS = ['Total items', 'reviews']
    f = open(filename, 'r')
    lines = f.readlines()
    entry = {}
    categories = []
    reviews = []
    similar_items = []
  
    for line in lines[0:total]:
        colonPos = line.find(':')

        if line.startswith("Id"):
            if reviews:
                entry["reviews"] = reviews
            if categories:
                entry["categories"] = categories
            yield entry
            entry = {}
            categories = []
            reviews = []
            rest = line[colonPos+2:]
            entry["id"] = rest[1:-1]
      
        elif line.startswith("similar"):
            similar_items = line.split()[2:]
            entry['similar_items'] = similar_items

    # "cutomer" is typo of "customer" in original data
        elif line.find("cutomer:") != -1:
            review_info = line.split()
            reviews.append({'customer_id': review_info[2], 
                          'rating': int(review_info[4]), 
                          'votes': int(review_info[6]), 
                          'helpful': int(review_info[8])})

        elif line.startswith("|"):
            categories.append(line)

        elif colonPos != -1:
            eName = line[:colonPos]
            rest = line[colonPos+2:]
            if not eName in IGNORE_FIELDS:
                if eName[0] == ' ':
                    eName = eName[2:]
                entry[eName] = rest[0:-1]

    if reviews:
        entry["reviews"] = reviews
    if categories:
        entry["categories"] = categories
    
    yield entry


file_path = "data/amazon-meta.txt"
line_num = sum(1 for line in open(file_path))
result = []
print(line_num)
for e in parse(file_path, total=line_num):
    if e:
        result.append(e)


15010574
{'id': ' 548551', 'ASIN': 'B00005MHUG', 'title': "That Travelin' Two-Beat/Sings the Great Country Hits", 'group': 'Music', 'salesrank': '0', 'similar': '5  B00008OETQ  B00005O6KL  B00006RY87  B0002OTI98  B0000634HG', 'categories': '6', 'reviews': [{'customer_id': 'ABTSEEYVYQ52M', 'rating': 5, 'votes': 9, 'helpful': 9}]}
